In [1]:
import os
os.chdir("GPT4/mutant/")

In [ ]:
import pandas as pd

# 
file_few = pd.read_excel("codellama_fewshot_predictions.xlsx")

questions_few = file_few['question']


In [ ]:
import pandas as pd

# 
file = pd.read_excel("codellama_zeroshot_predictions.xlsx")

questions = file['question']


In [ ]:
# predict  isCorrect
file_few['predict'] = ""
file_few['isCorrect'] = ""

# 
file.to_excel("codellama_fewshot_predictions.xlsx", index=False)

In [ ]:
# predict  isCorrect
file['predict'] = ""
file['isCorrect'] = ""

# 
file.to_excel("codellama_zeroshot_predictions.xlsx", index=False)

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
def load_plm(model_name):
  # AutoModelForMaskedLM
  tokenizer = AutoTokenizer.from_pretrained(f'huggingface/hub/{model_name}', trust_remote_code=True)
  model = AutoModelForCausalLM.from_pretrained(f'huggingface/hub/{model_name}', trust_remote_code=True)

  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  model.to(device)
  return tokenizer, model

In [ ]:
tokenizer, model = load_plm('CodeLlama-13b-Instruct-hf')

In [ ]:
def get_answer(question):
  input_ids = tokenizer.encode(question, return_tensors="pt").to(model.device)
    

  outputs = model.generate(input_ids, max_new_tokens=256, do_sample=False, top_p=0.9, no_repeat_ngram_size=2)
  response = tokenizer.decode(outputs[0])

  return response

In [ ]:
from transformers import pipeline

# 
chat_pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

# 
def chat_with_model(question):
    prompt_template = """ 
# <s>[INST] 
# {query}
# [/INST]
#  
"""
    
    prompt = prompt_template.format(query=question)
    input_ids = chat_pipe.tokenizer.encode(prompt, return_tensors='pt').to('cuda')
    response = chat_pipe.model.generate(input_ids, max_new_tokens=128, no_repeat_ngram_size=2)
    return chat_pipe.tokenizer.decode(response[0], skip_special_tokens=True)

# # 
# prompt = " Python ？"  # 
# print(chat_with_model(prompt))


In [ ]:
from transformers import pipeline


def get_answer(question):
  pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0)
   # 
  prompt_template = """ 
<s>[INST] <<SYS>>
You are a smart Mutant Test analyser. You should pay attention on the difference of the output.\n
<</SYS>>
{query}
[/INST]
"""
  prompt = prompt_template.format(query=question)

  outputs = pipe(prompt, max_new_tokens=256, no_repeat_ngram_size=2)

  # 
  return outputs[0]["generated_text"]

In [ ]:
from transformers import pipeline


def get_answer(question):
  pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0)
   # 
  prompt_template = "<|system|>\n<|end|>\n<|user|>\n{query}<|end|>\n<|assistant|>"
  prompt = prompt_template.format(query=question)

  outputs = pipe(prompt, max_new_tokens=256, no_repeat_ngram_size=2)

  # 
  return outputs[0]["generated_text"]

In [ ]:
question = """
Please analyze the two following provided code files in C or Java. Identify if they are semantically equal. 'Semantically equal' means two codes have the same meaning, that they have the same output given the same input.
Here are three semantically equal examples:  
The first example pair is
``` Code 1
double f(double M, double x) {
    x = (M + x) / 2;
    return x;
 }
```
``` Mutant Code 1
double f(double M, double x) {
    x = (M + x++ ) / 2;
    return x;
 }
```
Yes. The two codes are semantically euqal because `M + x++` first does `M + x` and then `x++`. Therefore, `(M + x) / 2` is the same with `(M + x++) / 2`.


The second example pair is 
``` Code 2
double f(int x, int divisor){
   return x*divisor;
}
```
```Mutant Code 2
double f(int x, int divisor){
    return x*divisor++;
}
```
Yes. The two codes are semantically euqal because `x*divisor` first does `x*divisor` and then `divisor++`. The two functions return the same values.


The third example pair is 
``` Code 3
int f(int a, int y, int x){
    int p = a;
    p = x + y;
    return p;
}
```
``` Mutant Code 3
int f(int a, int y, int x){
    int p = 1;
    p = x + y;
    return p;
}
```
Yes. The two codes are semantically euqal because the local value `p` is re-assigned by `x+y`. There, the change `int p = 1;` will not affect the function.

  

Please identify if the two following codes are semantically equal. Please only answer `yes` or `no`. `yes` means they are semantically equal. `no` means they are not. 
Input :
    ```Code 
// This is a mutant program.
// Author : ysma

public class Bisect
{

    double mEpsilon;

    double mNumber;

    double mResult;

    public Bisect()
    {
    }

    public  void setEpsilon( double epsilon )
    {
        this.mEpsilon = epsilon;
    }

    public  double sqrt( double N )
    {
        double x = N;
        double M = N;
        double m = 1;
        double r = x;
        double diff = x * x - N;
        while (Math.abs(diff) > mEpsilon) {
            if (diff < 0) {
                m = x;
                x = (M + x) / 2;
            } else {
                if (diff > 0) {
                    M = x;
                    x = (m + x) / 2;
                }
            }
            diff = x * x - N;
        }
        r = x;
        mResult = r;
        return r;
    }

}

```
```Mutant Code 
// This is a mutant program.
// Author : ysma

public class Bisect
{

    double mEpsilon;

    double mNumber;

    double mResult;

    public Bisect()
    {
    }

    public  void setEpsilon( double epsilon )
    {
        this.mEpsilon = epsilon;
    }

    public  double sqrt( double N )
    {
        double x = N;
        double M = N;
        double m = 1;
        double r = x;
        double diff = x * x - N;
        while (Math.abs(diff) > mEpsilon) {
            if (diff < 0) {
                m = x;
                x = (M + x) / 2;
            } else {
                if (diff > 0) {
                    M = x;
                    x = (m + x) / 2;
                }
            }
            diff = x * x - N;
        }
        r = x;
        mResult = -r;
        return r;
    }

}

```
The difference patch is 

    ```
    @@ -42 +42 @@
-        mResult = r;
+        mResult = -r;

    ```

In this example, I return r, so 'mResult = -r' and 'mResult=r' didn't affect the output, so mResult is an unused variable
"""

In [ ]:
ans = get_answer(question)

In [ ]:
print(ans)

In [ ]:
ans2 = chat_with_model(question)

In [ ]:
print(ans2)

In [ ]:
output = ans.split("[/INST]\n#")[-1]
print(output)

In [ ]:
def get_answer_tmp(question):
  return 'Yes'

In [3]:
from pandas import DataFrame
import pandas as pd

output_file = 'chatgpt4_zeroshot_predictions.xlsx'
output_data = DataFrame(pd.read_excel(output_file))

In [ ]:
for i, question in enumerate(questions):
  if i<=110:
    continue
  with open(f'question/{i}.txt', 'r') as f:
    question = f.read()
  print('------------------------')
  print(f"question:{i}")
  print(question.split("```\nThe difference patch is ")[-1])
  content = file.iloc[i].copy()
  ans = get_answer(question)
  output = ans.split("[/INST]\n#")[-1]
  print(output)

  with open(f'codellama_zero/{i}.txt', 'w') as f:
    f.write(output)


for i, question in enumerate(questions_few):
  if i<=110:
    continue
  with open(f'question/{i}.txt', 'r') as f:
    question = f.read()
  print('------------------------')
  print(f"question:{i}")
  print(question.split("```\nThe difference patch is ")[-1])
  content = file_few.iloc[i].copy()
  ans = get_answer(question)
  output = ans.split("[/INST]\n#")[-1]
  print(output)

  with open(f'codellama/{i}.txt', 'w') as f:
    f.write(output)

    

  # print(ans)
  # content['answer'] = ans

  # file.iloc[i] = content
  # file.to_excel(output_file, index=False)

In [5]:
# starchat
#  Excel 
import pandas as pd
import numpy as np
df = pd.read_excel('chatgpt4_zeroshot_predictions.xlsx')
for i in range(200):
    try:
        with open(f'zeroshot/{i}.txt') as f:
          content = f.read()
          #  content  "no" 
          if content.strip().lower().startswith("no"):
              predict = 0
          else:
              predict = 1
    except:
       content = 'no'

    # predict = 0

    #  predict  answer
    df.loc[i, 'predict'] = predict
    df.loc[i, 'answer'] = content

    #  label  predict
    df.loc[i, 'isCorrect'] = np.where(df.loc[i, 'label'] == df.loc[i, 'predict'], 'true', 'false')
  
#  Excel 
df.to_excel('chatgpt4_zeroshot_predictions.xlsx', index=False)

/tmp/ipykernel_3974698/801747223.py:25: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'false' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[i, 'isCorrect'] = np.where(df.loc[i, 'label'] == df.loc[i, 'predict'], 'true', 'false')
